In [4]:
#pip install metatrader5
!pip install pandas_datareader

     ------------------------------------ 109.5/109.5 kB 632.0 kB/s eta 0:00:00


# Criando Robô Pivot point 

In [5]:
import MetaTrader5 as mt5
from datetime import datetime
import time
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [6]:
# estabelecemos a conexão ao MetaTrader 5
# conecte-se ao MetaTrader 5
if not mt5.initialize():
    print("initialize() Falha ao Iniciar seu metra Trade 5")
    mt5.shutdown()

In [7]:
### Obtendo valores de ativos
#tenho que fazer uma estrutura de repecticao  para ficar gemrado os pontos de entrar e saida
def get_ohlc(ativo, timeframe, n=5):
    ativo = mt5.copy_rates_from_pos(ativo,timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit='s')
    ativo['Pivot'] = (ativo['high'] + ativo['low'] + ativo['close'])/3
    ativo['R1'] = 2*ativo['Pivot'] - ativo['low']
    ativo['S1'] = 2*ativo['Pivot'] - ativo['high']
    ativo['R2'] = ativo['Pivot'] + (ativo['high'] - ativo['low'])
    ativo['S2'] = ativo['Pivot'] - (ativo['high'] - ativo['low'])
    ativo['R3'] = ativo['Pivot'] + 2*(ativo['high'] - ativo['low'])
    ativo['S3'] = ativo['Pivot'] - 2*(ativo['high'] - ativo['low'])

    
    ativo.set_index('time', inplace = True)
    return ativo

In [8]:
ativo = (get_ohlc('ITUB4', mt5.TIMEFRAME_M5))

In [9]:
#verifica se está calculando 
ativo

,open,high,low,close,tick_volume,spread,real_volume,Pivot,R1,S1,R2,S2,R3,S3
time,,,,,,,,,,,,,,
2023-05-09 11:50:00,26.55,26.55,26.49,26.53,619,1,207000,26.523333,26.556667,26.496667,26.583333,26.463333,26.643333,26.403333
2023-05-09 11:55:00,26.52,26.54,26.52,26.52,134,1,34200,26.526667,26.533333,26.513333,26.546667,26.506667,26.566667,26.486667
2023-05-09 12:00:00,26.53,26.60,26.52,26.58,1092,1,337100,26.566667,26.613333,26.533333,26.646667,26.486667,26.726667,26.406667
2023-05-09 12:05:00,26.59,26.67,26.57,26.65,62,1,13900,26.630000,26.690000,26.590000,26.730000,26.530000,26.830000,26.430000
2023-05-09 12:10:00,26.65,26.70,26.65,26.70,2306,1,1334600,26.683333,26.716667,26.666667,26.733333,26.633333,26.783333,26.583333


### preparando a orden

In [53]:
#testando se o ativo é valido 
symbol = 'ITUB4'
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(symbol, "Não encontrato")
    mt5.shutdown()
    quit()

In [54]:
#adicionado symbol se nao existir
if not symbol_info.visible:
    print('Symbol Não visivel, tentnado adicionar')
    if not mt5.symbol_select(symbol,True):
        print('symbol_select({{}})failed, exit', symbol)
        mt5.shutdown()
        quit()

In [55]:
#preparando a ordem compra request  e ordem de venda

#########################################################
#preparando a ordem 
lot = 1.0
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).ask
desviation = 1
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "magic": 234000,
    "desviation": desviation,
    "comment": "prython script open",
    "type_time":mt5.ORDER_TIME_GTC,
    'type_filling':mt5.ORDER_FILLING_RETURN,
    
          }

In [56]:
#preparando a ordem de venda 
lot = 100.0
point = mt5.symbol_info(symbol).point
price=mt5.symbol_info_tick(symbol).bid
desviation = 1
request2={
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "price": price,
    
    "deviation": desviation,
    "magic": 234000,
    "comment": "python script close",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

In [ ]:
#ENVIO de Ordem buy, APENAS UM TESTE PARA SABER SE A COMUNICAÇÃO ENTRE PYTHON E MT5 ESTÁ CORRETA...
result = mt5.order_send(request)
#verificando a resultado da execulção 
print(f'1. Ordem enviada:{lot} de {symbol} ao preço de {price} com desvio de {desviation}',end  = '\r')

In [ ]:
# EXECULÇÃO DO BOT PIVOTPOINT 
# ANTES DE EXECUTAR ESSA CELULA CERTIFIQUE QUE NENHUMA ORDEM ESTEJA ABERTA.

tempo = time.time() + 18000
while time.time() < tempo:
    ativo = (get_ohlc('ITUB4', mt5.TIMEFRAME_M5))
    tick = mt5.symbol_info_tick('ITUB4')
    print (f'ITUB4 - ultimo valor: {tick.last}, Topo do Book C: {tick.bid},Topo do Book V: {tick.ask}', tick.last>ativo['Pivot'][-1 -1], end  = '\r')
    if tick.last> ativo['Pivot'][-1 -1]:
        if mt5.positions_get(symbol="ITUB4") == () or mt5.positions_get(symbol="ITUB4")[0][5] == 1:
            #enviadno ordem de compra 
            result = mt5.order_send(request)
            print(f'1. Ordem COMPRA enviada:{lot} de {symbol} ao preço de {price} com desvio de {desviation}',end  = '\r')

    if tick.last <ativo['S1'][-1 -1]:
        
        if mt5.positions_get(symbol="ITUB4") == () or mt5.positions_get(symbol="ITUB4")[0][5] == 0:
            #enviadno ordem de venda 
            result = mt5.order_send(request2)
        #verificando a resultado da execulção 
            print(f'1. Ordem VENDA enviada: {lot} de {symbol} ao preço de {price} com desvio de {desviation}',end  = '\r')
        
    time.sleep(0.5)